<a href="https://colab.research.google.com/github/Nilanshrajput/AdvitiyaHackathon/blob/master/BertV2_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 481kB 3.5MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 65.4MB/s eta 0:00:01
     |████████████████████████████████| 870kB 60.4MB/s 
     |████████████████████████████████| 3.1MB 48.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=1eaf2390782ced39936722d441525b542e7eadc5a5cdca9cb43e2dcbf80b6fc5
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [2]:
import torch
import numpy as np
import pandas as pd
from transformers import BertModel, BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from torch import nn

from time import time




In [ ]:

%%capture
from tqdm import tnrange
from tqdm import tqdm_notebook as tqdm




In [4]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
drive_pt="/content/drive/My Drive/dl_projects/AdvitiyaHackathon/"

git_path = "AdvitiyaHackathon/"
path = drive_pt+"ScrappeData"

In [ ]:
df1 = pd.read_csv(drive_pt+"claims_final_train_clean.csv",index_col=0)
df2 = pd.read_csv(drive_pt+"claims_final_val_clean.csv", index_col= 0 )

In [41]:
df1.head()

,text,cat
0,agricultural autonomous vehicle platform arti...,3
1,drive circuit semiconductor type light source...,2
2,modular electronic carbonator fluid level con...,0
3,natural stable solution alpha acid use improv...,1
4,par bake mill coffee bean use food beverage d...,0


In [ ]:
df = df1.append(df2, ignore_index =True).sample(frac=1).reset_index(drop=True)


In [ ]:
"""df1 = pd.read_csv(drive_pt+"claims_final_val_clean.csv", index_col= 0 ).sample(frac=1).reset_index(drop=True)
df2 = pd.read_csv(drive_pt+"abstract_final_val_clean.csv", index_col= 0 ).sample(frac=1).reset_index(drop=True)
df3 = pd.read_csv(drive_pt+"classification_final_val_clean.csv", index_col= 0 ).sample(frac=1).reset_index(drop=True)"""

In [ ]:
#df_val_all = df1.append([df2,df3], ignore_index=True).sample(frac=1).reset_index(drop=True)

In [ ]:
#df_val_all.to_csv(drive_pt+"train/val_all.csv")

In [ ]:
"""df1=pd.read_csv(drive_pt+"train/claims_train_all.csv",index_col=0).sample(frac=1).reset_index(drop=True)
df2=pd.read_csv(drive_pt+"train/classification_train_all.csv",index_col= 0).sample(frac=1).reset_index(drop=True)
df3=pd.read_csv(drive_pt+"train/abstract_train_all.csv",index_col= 0).sample(frac=1).reset_index(drop=True)
"""

In [ ]:
#df = df1.append([df2,df3],ignore_index= True).sample(frac=1).reset_index(drop=True)

In [ ]:
#df.to_csv(drive_pt+"train/all_text_train_all.csv")

In [ ]:
df1 = df1.sample(frac=1).reset_index(drop=True)
df2 = df2.sample(frac=1).reset_index(drop=True)


In [ ]:
"""df1.to_csv(drive_pt+"final_train_clean.csv")
df2.to_csv(drive_pt+"final_val_clean.csv")"""

'df1.to_csv(drive_pt+"final_train_clean.csv")\ndf2.to_csv(drive_pt+"final_val_clean.csv")'

In [ ]:
#df2.head()

In [ ]:
bert_model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:

from keras.preprocessing.sequence import pad_sequences


class LoadDataset(Dataset):

    def __init__(self, filename, maxlen):

        # Store the contents of the file in a pandas dataframe
        self.df = pd.read_csv(filename, delimiter=',',index_col=0)

        # Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        # Define the Maxlength for padding/truncating
        self.maxlen = maxlen

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        # Selecting the sentence and label at the specified index in the data frame
        sentence = self.df.loc[index, 'text']
        sentence = sentence[20:]
        #sentence = re.sub(' +', ' ',sentence)
        label = torch.tensor((self.df.loc[index, 'cat']))

        # Tokenize the sentence
        tokens_ids = self.tokenizer.encode(str(sentence),add_special_tokens = True,max_length = self.maxlen, pad_to_max_length=True)
        """
        tokens_ids = pad_sequences([tokens_ids], maxlen=self.maxlen, dtype="long", 
                          value=0, truncating="post", padding="post")
        """   
        # Inserting the CLS and SEP token at the beginning and end of the sentence
        #tokens = ['[CLS]'] + tokens 
        """
        if len(tokens_ids) < self.maxlen:
            tokens_ids = [tokens_ids + [0] for _ in range(self.maxlen - len(tokens_ids))]
        else:
            tokens_ids = tokens_ids[:self.maxlen-1]         
        """  
        # Padding/truncating the sentences to the maximum length


        # Convert the sequence to ids with BERT Vocabulary
        #tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        
        # Converting the list to a pytorch tensor
        tokens_ids_tensor = torch.tensor(tokens_ids)

        # Obtaining the attention mask
        attn_mask = (tokens_ids_tensor != 0).long()

        return tokens_ids_tensor, attn_mask, label

In [ ]:
# Creating instances of training and validation set
train_set = LoadDataset(filename = drive_pt+'train/all_text_train_all.csv', maxlen = 128)
val_set = LoadDataset(filename = drive_pt+'train/val_all.csv', maxlen = 128)


In [ ]:

# Creating intsances of training and validation dataloaders
train_loader = DataLoader(train_set, batch_size = 32, shuffle = True, num_workers = 5)
val_loader = DataLoader(val_set, batch_size = 32, shuffle = True, num_workers = 5)

In [ ]:
#batch = next(iter(train_loader))

In [ ]:
#batch[0].shape

In [ ]:

class PatentClassifier(nn.Module):

    def __init__(self, freeze_bert = True):
        super(PatentClassifier, self).__init__()

        # Instantiating the BERT model object 
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        
        # Defining layers like dropout and linear
        self.dropout = nn.Dropout(0.15)
        self.classifier = nn.Linear(768, 4)

    def forward(self, seq, attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        # Getting contextualized representations from BERT Layer
        cont_reps, _ = self.bert_layer(seq, attention_mask = attn_masks)

        # Obtaining the representation of [CLS] head
        cls_rep = cont_reps[:, 0]
        # print('CLS shape: ',cls_rep.shape)

        # Feeding cls_rep to the classifier layer
        logits = self.classifier(cls_rep)
        # print('Logits shape: ',logits.shape)

        return logits
        
        

In [53]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 4, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = True, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

#model = PatentClassifier(freeze_bert= False)

# Tell pytorch to run this model on the GPU.
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [54]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 3e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [ ]:

from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 30

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_loader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [59]:


import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [17]:
# Saving our model
import os
save_path = drive_pt+ "Models/"


"""# Loading the checkpoints for resuming training
checkpoint = torch.load(save_path+'finalbertv4_claims_512_8_20.pth')
model.load_state_dict(checkpoint['model_state_dict'])"""

"# Loading the checkpoints for resuming training\ncheckpoint = torch.load(save_path+'finalbertv4_claims_512_8_20.pth')\nmodel.load_state_dict(checkpoint['model_state_dict'])"

In [60]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_loader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_loader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
       
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_loader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in val_loader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 30 ========
Training...
  Batch    40  of    113.    Elapsed: 0:00:47.
  Batch    80  of    113.    Elapsed: 0:01:32.

  Average training loss: 0.46
  Training epcoh took: 0:02:09

Running Validation...
  Accuracy: 0.95
  Validation took: 0:00:07

======== Epoch 2 / 30 ========
Training...
  Batch    40  of    113.    Elapsed: 0:00:49.
  Batch    80  of    113.    Elapsed: 0:01:34.

  Average training loss: 0.12
  Training epcoh took: 0:02:08

Running Validation...
  Accuracy: 0.99
  Validation took: 0:00:07

======== Epoch 3 / 30 ========
Training...
  Batch    40  of    113.    Elapsed: 0:00:51.
  Batch    80  of    113.    Elapsed: 0:01:35.

  Average training loss: 0.07
  Training epcoh took: 0:02:08

Running Validation...
  Accuracy: 0.99
  Validation took: 0:00:07

======== Epoch 4 / 30 ========
Training...
  Batch    40  of    113.    Elapsed: 0:00:49.
  Batch    80  of    113.    Elapsed: 0:01:34.

  Average training loss: 0.05
  Training epcoh took: 0:02:07

KeyboardInterrupt: ignored

In [ ]:
# Saving our model
import os
save_path = drive_pt+ "Models/"
if not os.path.isdir(save_path):
    os.mkdir(save_path)

torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, save_path+'all_final_calims_128_32_30.pth')

ERROR: Could not find a version that satisfies the requirement bertviz (from versions: none)
ERROR: No matching distribution found for bertviz


In [ ]:
# Creating instances of training and validation set
train_set = LoadDataset(filename = drive_pt+'train/abstract_train_all.csv', maxlen = 512)
val_set = LoadDataset(filename = drive_pt+'classification_final_val_clean.csv', maxlen = 512)
# Creating intsances of training and validation dataloaders
train_loader = DataLoader(train_set, batch_size = 8, shuffle = False, num_workers = 5)
val_loader = DataLoader(val_set, batch_size = 8, shuffle = False, num_workers = 5)

Evaluation

In [73]:
# Saving our model
import os
save_path = drive_pt+ "Models/"
model1,model2,model3,model4,model5 = model,model,model,model,model

# Loading the checkpoints for resuming training
checkpoint = torch.load(save_path+'finalbertv5_abstract_512_8_20.pth')
model3.load_state_dict(checkpoint['model_state_dict'])

# Loading the checkpoints for resuming training
checkpoint = torch.load(save_path+'bertv6_classification_512_8_20.pth')
model2.load_state_dict(checkpoint['model_state_dict'])

# Loading the checkpoints for resuming training
checkpoint = torch.load(save_path+'finalbertv4_claims_512_8_20.pth')
model3.load_state_dict(checkpoint['model_state_dict'])

# Loading the checkpoints for resuming training
checkpoint = torch.load(save_path+'all_final_calims_128_32_30.pth')
model4.load_state_dict(checkpoint['model_state_dict'])

# Loading the checkpoints for resuming training
checkpoint = torch.load(save_path+'all_final_calims_300_16_20.pth')
model5.load_state_dict(checkpoint['model_state_dict'])




<All keys matched successfully>

In [ ]:
"""model1.eval()
model2.eval()
model3.eval()
"""

In [ ]:
def most_frequent(List): 
    counter = 0
    num = List[0] 
      
    for i in List: 
        curr_frequency = List.count(i) 
        if(curr_frequency> counter): 
            counter = curr_frequency 
            num = i 
  
    return num 



def return_model_output(model, sentences, dev ):
  results=[]
  model.eval()
  for sentence in sentences:
    b_input_ids, b_input_mask = pre_process(sentence, maxlen = 512,device=dev)
    output = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
    logits = output[0]
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    cat = np.argmax(logits, axis=1).flatten()
    results.append(int(cat))

  return results

def pre_process(sentence, maxlen,device):
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  #sentence = sentence[50:]
  tokens_ids = tokenizer.encode(str(sentence),add_special_tokens = True,max_length = maxlen, pad_to_max_length=True)
  # Converting the list to a pytorch tensor
  tokens_ids_tensor = torch.tensor(tokens_ids)

  # Obtaining the attention mask
  attn_mask = (tokens_ids_tensor != 0).long()

  return tokens_ids_tensor.unsqueeze(dim = 0).to(device), attn_mask.unsqueeze(dim = 0).to(device)


In [ ]:
df1 = pd.read_csv(drive_pt+"claims_final_test_clean.csv", index_col= 0 )
df2 = pd.read_csv(drive_pt+"classification_final_test_clean.csv", index_col= 0 )
df3 = pd.read_csv(drive_pt+"abstract_final_test_clean.csv", index_col= 0 )

In [76]:


for index in tnrange(0,len(df1)):
  
  sentence1 = df1.loc[index, 'text']
  sentence2 = df2.loc[index, 'text']
  sentence3 = df3.loc[index, 'text']
  sentences = [sentence1, sentence2, sentence3]
  results = return_model_output(model1, sentences= sentences,dev = device)
  results += return_model_output(model2, sentences= sentences,dev = device)
  results += return_model_output(model3, sentences= sentences,dev = device)
  results += return_model_output(model4, sentences= sentences,dev = device)
  results += return_model_output(model5, sentences= sentences,dev = device)

  final_cat = most_frequent(results)
  df1.loc[index, 'cat'] = final_cat


In [ ]:
#dftest2 = pd.read_csv(drive_pt+"Cleaned_data/test.csv")

In [ ]:
#dftest2
#dftest2.to_csv("sub2.csv")

In [ ]:
dftest = pd.read_csv(drive_pt+"test.csv")

In [ ]:
dftest["Category"] = df1.iloc[:,1]

In [28]:
dftest

,Publication Number,Category
0,US20150093479A1,0
1,US20150086688A1,0
2,US20150086675A1,0
3,US20150086655A1,0
4,US20150080469A1,0
...,...,...
195,US20160068103A1,3
196,US20160046358A1,3
197,EP2969657A4,3
198,EP2988991A1,3


In [ ]:
#dftest=pd.read_csv("sub1.csv",index_col=0)

In [ ]:
names = ["Non-Alcohol","Alcohol", "Non-Autonomous Vehicles", "Autonomous Vehicles"]
for i in range(0,4):
  mask = dftest.Category == i
  column_name = 'Category'
  dftest.loc[mask, column_name] = names[i]

In [85]:
dftest.loc[34, "Category"]

'Non-Alcohol'

In [ ]:
dftest.to_csv("submission_model_all5.csv")

In [147]:
"""index =150
sentence1 = df1.loc[index, 'text']
sentence2 = df2.loc[index, 'text']
sentence3 = df3.loc[index, 'text']
sentences = [sentence1, sentence2, sentence3]
results1 = return_model_output(model1, sentences= sentences,dev = device)
print(results1)"""

[array([3]), array([3]), array([3])]


**Garbage Code Below**

' frame unman grind vehicle ugv frame unman grind vehicle say frame comprise pair self propel lateral module extend longitudinal direction mutually removably couple transverse direction say self propel lateral module assemble form support structure central module removably couple say frame provide control system configure control movement actuation say self propel lateral module little one say self propel lateral module comprise pair bear portion connection portion mutually constrain say bear portion permit relative movement thereof extend configuration say bear portion define great longitudinal extension say self propel lateral module compact configuration say bear portion define small longitudinal extension say self propel lateral module say self propel lateral module assemble form support structure central module removably couple say frame provide control system configure control movement actuation say self propel lateral module little one say self propel lateral module comprise pai

In [130]:
label

0

In [ ]:
mask = dftest.Category == 3
column_name = 'Category'
dftest.loc[mask, column_name] = "Autonomous Vehicles"

In [ ]:
dftest.to_csv("submission.csv")

In [101]:
[1]+[2]

[1, 2]

In [ ]:
b_input_ids, b_input_mask = pre_process(sentence1, maxlen = 512,device=device)

In [142]:
b_input_mask = b_input_mask
b_input_ids = b_input_ids
b_input_mask.shape

torch.Size([1, 512])

In [ ]:
model.eval()

In [ ]:
outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)

In [ ]:
logits = outputs[0]

# Move logits and labels to CPU
logits = logits.detach().cpu().numpy()
a=np.argmax(logits, axis=1).flatten()

In [145]:
a

array([3])

In [33]:
int(a)

1

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

**Ignore cells below**

In [ ]:
import re
location = "d125sd{g5d"
letters_only = re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          str(location))

In [ ]:
letters_only

'd   sd g d'

In [ ]:
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

print(device)

cuda


In [ ]:
# Defining a function for calculating accuracy
def logits_accuracy(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    preds = (probs > 0.5).long()
    acc = (preds.squeeze() == labels).float().mean()
    return acc

In [ ]:
# Defining an evaluation function for training 
def evaluate(net, criterion, val_loader, device):
  
    losses, accuracies = 0, 0
    
    # Setting model to evaluation mode
    net.eval()

    count = 0
    for (seq, attn_masks, labels) in val_loader:
        count += 1

        # Move inputs and targets to device
        seq, attn_masks, labels = seq.to(device), attn_masks.to(device), labels.to(device)

        # Get logit predictions
        val_logits = net(seq, attn_masks)

        # Calculate loss
        val_loss = criterion(val_logits.squeeze(-1), labels.float())
        losses += val_loss.item()

        # Calculate validation accuracy
        accuracies += logits_accuracy(val_logits, labels)

    return losses / count, accuracies / count

In [ ]:
def train(net, criterion, optimizer, train_loader, val_loader, device, epochs=4, print_every=100):
    
    # Move model to device
    net.to(device)
    # Setting model to training mode
    net.train()

    print('========== ========== STARTING TRAINING ========== ==========')

    for epoch in range(epochs):

        print('\n\n========== EPOCH {} =========='.format(epoch))        
        t1 = time()

        for i, (seq, attn_masks, labels) in enumerate(train_loader):

            # Clear gradients
            optimizer.zero_grad()  

            # Moving tensors to device
            seq, attn_masks, labels = seq.to(device), attn_masks.to(device), labels.to(device)

            # Obtaining the logits from the model
            logits = net(seq,attn_masks)

            # Calculating the loss
            loss = criterion(logits.squeeze(-1), labels.float())

            # Backpropagating the gradients
            loss.backward()

            # Clipping gradients to tackle exploding gradients
            nn.utils.clip_grad_norm_(net.parameters(), 1)

            # Optimization step
            optimizer.step()

            if (i + 1) % print_every == 0:
                print("Iteration {} ==== Loss: {}".format(i+1, loss.item()))

        t2 = time()
        print('Time Taken for Epoch: {}'.format(t2-t1))
        print('\n========== Validating ==========')
        mean_val_loss, mean_val_acc = evaluate(net, criterion, val_loader, device)
        print("Validation Loss: {}\nValidation Accuracy: {}".format(mean_val_loss, mean_val_acc))
